## 🧠 Chapter 03: Data Driven Decision Making with advanced SQL queries

### 🔍 Nested Queries

Nested queries allow you to embed one query inside another using the result of the **inner query** in the `WHERE` or `HAVING` clause of the **outer query**.

#### Example: Customers who rated movies ≤ 3

**Step 1: Inner query**

```sql
SELECT DISTINCT customer_id 
FROM renting 
WHERE rating <= 3;
```

<table>
<thead><tr><th>customer_id</th></tr></thead>
<tbody>
<tr><td>28</td></tr>
<tr><td>41</td></tr>
<tr><td>86</td></tr>
<tr><td>120</td></tr>
</tbody>
</table>

**Step 2: Outer query**

```sql
SELECT name 
FROM customers 
WHERE customer_id IN 
    (SELECT DISTINCT customer_id 
     FROM renting 
     WHERE rating <= 3);
```

<table>
<thead><tr><th>name</th></tr></thead>
<tbody>
<tr><td>Sidney Généreux</td></tr>
<tr><td>Zara Mitchell</td></tr>
</tbody>
</table>



### 🧾 Nested Query in HAVING

**Inner query:**

```sql
SELECT MIN(date_account_start) 
FROM customers 
WHERE country = 'Austria';
```

<table>
<thead><tr><th>min</th></tr></thead>
<tbody>
<tr><td>2017-11-22</td></tr>
</tbody>
</table>

**Outer query:**

```sql
SELECT country, MIN(date_account_start) 
FROM customers 
GROUP BY country 
HAVING MIN(date_account_start) < 
    (SELECT MIN(date_account_start) 
     FROM customers 
     WHERE country = 'Austria');
```

<table>
<thead><tr><th>country</th><th>min</th></tr></thead>
<tbody>
<tr><td>Spain</td><td>2017-02-14</td></tr>
<tr><td>Great Britain</td><td>2017-03-31</td></tr>
</tbody>
</table>



### 🎭 Querying by Movie Title

```sql
SELECT name 
FROM actors 
WHERE actor_id IN 
    (SELECT actor_id  
     FROM actsin 
     WHERE movie_id = 
         (SELECT movie_id  
          FROM movies 
          WHERE title = 'Ray'));
```

<table>
<thead><tr><th>name</th></tr></thead>
<tbody>
<tr><td>Jamie Foxx</td></tr>
<tr><td>Kerry Washington</td></tr>
<tr><td>Regina King</td></tr>
</tbody>
</table>



## 🔄 Correlated Nested Queries

These depend on values from the **outer query**. Each row from the outer query affects the result of the inner query.

### 🎬 Example: Movies rented more than 5 times

```sql
SELECT * 
FROM movies AS m 
WHERE 5 <  
    (SELECT COUNT(*) 
     FROM renting AS r 
     WHERE r.movie_id = m.movie_id);
```

<table>
<thead><tr><th>movie_id</th><th>title</th><th>genre</th><th>runtime</th><th>year_of_release</th><th>renting_price</th></tr></thead>
<tbody>
<tr><td>1</td><td>One Night at McCool's</td><td>Comedy</td><td>93</td><td>2001</td><td>2.09</td></tr>
<tr><td>2</td><td>Swordfish</td><td>Drama</td><td>99</td><td>2001</td><td>2.19</td></tr>
</tbody>
</table>



### 🍿 Movies rented less than 5 times

```sql
SELECT * 
FROM movies AS m 
WHERE 5 >  
    (SELECT COUNT(*) 
     FROM renting AS r 
     WHERE r.movie_id = m.movie_id);
```

<table>
<thead><tr><th>movie_id</th><th>title</th><th>genre</th><th>runtime</th><th>year_of_release</th><th>renting_price</th></tr></thead>
<tbody>
<tr><td>17</td><td>The Human Stain</td><td>Mystery & Suspense</td><td>106</td><td>2003</td><td>1.99</td></tr>
<tr><td>20</td><td>Love Actually</td><td>Comedy</td><td>135</td><td>2003</td><td>2.29</td></tr>
</tbody>
</table>



## ✅ EXISTS and NOT EXISTS

These are **correlated** queries that return `TRUE` or `FALSE`.

* `EXISTS` → at least one row matches
* `NOT EXISTS` → no rows match

### 🎬 Movies with at least one rating

```sql
SELECT * 
FROM movies AS m 
WHERE EXISTS 
    (SELECT * 
     FROM renting AS r 
     WHERE rating IS NOT NULL 
     AND r.movie_id = m.movie_id);
```

<table>
<thead><tr><th>movie_id</th><th>title</th><th>genre</th><th>runtime</th><th>year_of_release</th><th>renting_price</th></tr></thead>
<tbody>
<tr><td>1</td><td>One Night at McCool's</td><td>Comedy</td><td>93</td><td>2001</td><td>2.09</td></tr>
<tr><td>2</td><td>Swordfish</td><td>Drama</td><td>99</td><td>2001</td><td>2.19</td></tr>
</tbody>
</table>



### 🚫 Movies with no ratings (`NOT EXISTS`)

```sql
SELECT * 
FROM movies AS m 
WHERE NOT EXISTS 
    (SELECT * 
     FROM renting AS r 
     WHERE rating IS NOT NULL 
     AND r.movie_id = m.movie_id);
```

<table>
<thead><tr><th>movie_id</th><th>title</th><th>genre</th><th>runtime</th><th>year_of_release</th><th>renting_price</th></tr></thead>
<tbody>
<tr><td>11</td><td>Showtime</td><td>Comedy</td><td>95</td><td>2002</td><td>1.79</td></tr>
</tbody>
</table>



## 🧩 Set Operations: UNION and INTERSECT

Use these when combining results from multiple queries.

### 🔗 `UNION`: Combine without duplicates

```sql
SELECT title, genre, renting_price 
FROM movies 
WHERE renting_price > 2.8 
UNION 
SELECT title, genre, renting_price 
FROM movies 
WHERE genre = 'Action & Adventure';
```

<table>
<thead><tr><th>title</th><th>genre</th><th>renting_price</th></tr></thead>
<tbody>
<tr><td>Fool's Gold</td><td>Action & Adventure</td><td>2.69</td></tr>
<tr><td>Astro Boy</td><td>Action & Adventure</td><td>2.89</td></tr>
<tr><td>Fair Game</td><td>Drama</td><td>2.89</td></tr>
</tbody>
</table>



### 🔍 `INTERSECT`: Keep only overlapping results

```sql
SELECT title, genre, renting_price 
FROM movies 
WHERE renting_price > 2.8 
INTERSECT 
SELECT title, genre, renting_price 
FROM movies 
WHERE genre = 'Action & Adventure';
```

<table>
<thead><tr><th>title</th><th>genre</th><th>renting_price</th></tr></thead>
<tbody>
<tr><td>Astro Boy</td><td>Action & Adventure</td><td>2.89</td></tr>
</tbody>
</table>

## ✅ Summary

| Feature                 | Purpose                                                |
| ----------------------- | ------------------------------------------------------ |
| Nested Queries          | Use inner results for filtering in `WHERE` or `HAVING` |
| Correlated Queries      | Inner query depends on the outer query’s current row   |
| `EXISTS` / `NOT EXISTS` | Check for presence/absence of matching rows            |
| `UNION` / `INTERSECT`   | Combine query results (with/without duplicates)        |
